# 📄 PDF Chunking & Cleaning - Sederhana & Efektif

## 1. Import Libraries

In [88]:
import pdfplumber
import pandas as pd
import re
import os

## 2. Ekstraksi Teks dari PDF

In [ ]:
# Path ke file PDF Anda
pdf_file_path = 'dataset/Template Proposal.pdf'

# Ekstrak teks dari PDF menggunakan pdfplumber
full_text = ""

print("📖 Mengekstrak PDF dengan pdfplumber...")
with pdfplumber.open(pdf_file_path) as pdf:
    total_pages = len(pdf.pages)
    print(f"📄 Total halaman: {total_pages}")
    
    for page_num, page in enumerate(pdf.pages):
        text = page.extract_text()
        if text:
            full_text += text + "\n"
        
        if (page_num + 1) % 5 == 0 or page_num == 0:
            print(f"   ✓ Halaman {page_num + 1}/{total_pages}")

print(f"\n✅ Ekstraksi selesai!")
print(f"📊 Total karakter: {len(full_text):,}")
print(f"📊 Total kata:  {len(full_text.split()):,}")

📖 Mengekstrak PDF dengan pdfplumber...
📄 Total halaman: 7
   ✓ Halaman 1/7
   ✓ Halaman 5/7

✅ Ekstraksi selesai!
📊 Total karakter: 2,490
📊 Total kata:  349


In [90]:
# Preview teks mentah
print("\n📝 Preview teks SEBELUM cleaning:")
print("="*80)
print(full_text[:800])


📝 Preview teks SEBELUM cleaning:
PROPOSAL BISNIS STARTUP
[NAMA STARTUP ANDA]
Diajukan Untuk:
Program Inkubasi Startup - Universitas Komputer Indonesia
[Logo Unikom]
Disusun Oleh (Tim):
1. Nama Ketua Tim - NIM - Prodi
2. Nama Anggota 1 - NIM - Prodi
3. Nama Anggota 2 - NIM - Prodi
KOTA BANDUNG
TAHUN 2025
DAFTAR ISI
1. BAB I: PENDAHULUAN
2. BAB II: DESKRIPSI USAHA
3. BAB III: ASPEK KEUANGAN
4. BAB IV: PENUTUP
5. LAMPIRAN
BAB I: PENDAHULUAN
1.1 Latar Belakang Usaha
Petunjuk Pengisian:
Jelaskan masalah utama yang melatarbelakangi terbentuknya usaha ini. Sertakan
data atau fakta pendukung mengapa masalah ini penting untuk diselesaikan.
BAB II: DESKRIPSI USAHA
2.1 Noble Purpose (Tujuan Mulia)
Petunjuk Pengisian:
Apa tujuan mulia dari bisnis ini selain mencari keuntungan? Dampak positif apa
yang ingin diberikan kepada masyarakat


## 3. Pembersihan Teks (Deep Cleaning)

In [91]:
def deep_clean_text(text:  str) -> str:
    """
    Pembersihan teks yang comprehensive
    """
    print("🧹 Cleaning text...")
    
    # 1. Normalisasi line breaks
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    
    # 2. Hapus multiple spaces (termasuk tab)
    text = re.sub(r'[ \t]+', ' ', text)
    
    # 3. Hapus spasi di awal/akhir setiap baris
    lines = [line.strip() for line in text.split('\n')]
    text = '\n'.join(lines)
    
    # 4. Hapus baris kosong berlebihan
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # 5. Fix spasi sebelum tanda baca
    text = re.sub(r'\s+([.,;: ! ?])', r'\1', text)
    
    # 6. Fix spasi setelah tanda baca
    text = re.sub(r'([.,;:!?])([A-Za-z])', r'\1 \2', text)
    
    # 7. Hapus multiple dots
    text = re.sub(r'\.{2,}', '.', text)
    
    # 8. Fix encoding
    text = text.encode('utf-8', 'ignore').decode('utf-8')
    
    # 9. Trim
    text = text.strip()
    
    print("✅ Cleaning selesai")
    return text


In [92]:
cleaned_text = deep_clean_text(full_text)

print(f"📊 Sebelum cleaning: {len(full_text):,} karakter")
print(f"📊 Setelah cleaning: {len(cleaned_text):,} karakter")

🧹 Cleaning text...
✅ Cleaning selesai
📊 Sebelum cleaning: 2,490 karakter
📊 Setelah cleaning: 2,489 karakter


In [93]:
# Preview teks setelah cleaning
print("\n📝 Preview teks SETELAH cleaning:")
print("="*80)
print(cleaned_text[:800])


📝 Preview teks SETELAH cleaning:
PROPOSAL BISNIS STARTUP
[NAMA STARTUP ANDA]
Diajukan Untuk:
Program Inkubasi Startup - Universitas Komputer Indonesia
[Logo Unikom]
Disusun Oleh (Tim):
1. Nama Ketua Tim - NIM - Prodi
2. Nama Anggota 1 - NIM - Prodi
3. Nama Anggota 2 - NIM - Prodi
KOTA BANDUNG
TAHUN 2025
DAFTAR ISI
1. BAB I: PENDAHULUAN
2. BAB II: DESKRIPSI USAHA
3. BAB III: ASPEK KEUANGAN
4. BAB IV: PENUTUP
5. LAMPIRAN
BAB I: PENDAHULUAN
1.1 Latar Belakang Usaha
Petunjuk Pengisian:
Jelaskan masalah utama yang melatarbelakangi terbentuknya usaha ini. Sertakan
data atau fakta pendukung mengapa masalah ini penting untuk diselesaikan.
BAB II: DESKRIPSI USAHA
2.1 Noble Purpose (Tujuan Mulia)
Petunjuk Pengisian:
Apa tujuan mulia dari bisnis ini selain mencari keuntungan? Dampak positif apa
yang ingin diberikan kepada masyarakat


## 4. Deteksi Header/Sub Judul dalam Dokumen

In [94]:
def analyze_document_structure(text: str):
    """
    Menganalisis struktur dokumen
    """
    lines = text.split('\n')
    structure = []
    
    main_header_pattern = r'^(\d+)\.\s*(.+?)(?:\s*: )?$'
    sub_header_pattern = r'^-\s*(.+)$'
    
    for i, line in enumerate(lines):
        line_stripped = line.strip()
        
        # Cek main header
        main_match = re.match(main_header_pattern, line_stripped)
        if main_match: 
            structure.append({
                'line':  i,
                'type': 'main_header',
                'number': main_match.group(1),
                'title': main_match.group(2).strip(),
                'full_text': line_stripped
            })
            continue
        
        # Cek sub header
        sub_match = re.match(sub_header_pattern, line_stripped)
        if sub_match:
            structure.append({
                'line': i,
                'type':  'sub_header',
                'title': sub_match.group(1).strip(),
                'full_text': line_stripped
            })
    
    return structure

In [95]:
structure = analyze_document_structure(cleaned_text)

print(f"🔍 Struktur dokumen ditemukan: {len(structure)} headers")
print("="*80)
for item in structure[: 20]: 
    if item['type'] == 'main_header':
        print(f"\n{item['number']}. {item['title']}")
    else:
        print(f"   - {item['title']}")

🔍 Struktur dokumen ditemukan: 22 headers

1. Nama Ketua Tim - NIM - Prodi

2. Nama Anggota 1 - NIM - Prodi

3. Nama Anggota 2 - NIM - Prodi

1. BAB I: PENDAHULUAN

2. BAB II: DESKRIPSI USAHA

3. BAB III: ASPEK KEUANGAN

4. BAB IV: PENUTUP

5. LAMPIRAN

1. 1 Latar Belakang Usaha

2. 1 Noble Purpose (Tujuan Mulia)

2. 2 Identifikasi Konsumen Potensial

2. 3 Produk Inovatif

2. 4 Strategi Pemasaran

2. 5 Sumber Daya yang Dimiliki

3. 1 Laporan/Proyeksi Keuangan

3. 2 Rencana Anggaran Belanja (RAB) Produksi

4. 1 Kesimpulan

1. Business Model Canvas (BMC)

2. File Excel Laporan Keuangan

3. File Excel RAB


## 5. Chunking Berdasarkan Header

In [96]:
def chunk_by_sub_headers(text:  str) -> pd.DataFrame:
    """
    Chunking per sub header - Setiap sub header jadi chunk terpisah
    """
    print("📑 Chunking per sub header (granular)...")
    
    lines = text.split('\n')
    chunks = []
    chunk_id = 1
    
    main_header_pattern = r'^(\d+)\.\s*(.+?)(?:\s*:)?$'
    sub_header_pattern = r'^-\s*(. +)$'
    
    current_main_number = None
    current_main_title = None
    current_sub_title = None
    current_content = []
    
    intro_content = []
    first_main_header_found = False
    
    for i, line in enumerate(lines):
        line_stripped = line.strip()
        
        if not line_stripped:
            continue
        
        # Cek main header
        main_match = re. match(main_header_pattern, line_stripped)
        if main_match:
            # Simpan chunk sebelumnya
            if current_sub_title and current_content: 
                content_text = ' '.join([l.strip() for l in current_content if l.strip()])
                content_text = ' '.join(content_text.split())
                
                if content_text:
                    chunks.append({
                        'chunk_id': chunk_id,
                        'main_header_number': current_main_number,
                        'main_header_title': current_main_title,
                        'sub_header_title': current_sub_title,
                        'content': content_text
                    })
                    chunk_id += 1
            
            elif current_main_title and current_content and not current_sub_title:
                content_text = ' '.join([l.strip() for l in current_content if l.strip()])
                content_text = ' '.join(content_text.split())
                
                if content_text:
                    chunks.append({
                        'chunk_id': chunk_id,
                        'main_header_number': current_main_number,
                        'main_header_title': current_main_title,
                        'sub_header_title': None,
                        'content': content_text
                    })
                    chunk_id += 1
            
            # Update main header
            current_main_number = main_match.group(1)
            current_main_title = main_match.group(2).strip()
            current_sub_title = None
            current_content = []
            first_main_header_found = True
            continue
        
        # Cek sub header
        sub_match = re.match(sub_header_pattern, line_stripped)
        if sub_match:
            # Simpan chunk sebelumnya
            if current_sub_title and current_content:
                content_text = ' '.join([l.strip() for l in current_content if l.strip()])
                content_text = ' '.join(content_text.split())
                
                if content_text: 
                    chunks.append({
                        'chunk_id': chunk_id,
                        'main_header_number': current_main_number,
                        'main_header_title': current_main_title,
                        'sub_header_title': current_sub_title,
                        'content': content_text
                    })
                    chunk_id += 1
            
            # Update sub header
            current_sub_title = sub_match.group(1).strip()
            current_content = []
            continue
        
        # Ini adalah content
        if not first_main_header_found: 
            intro_content.append(line)
        elif current_sub_title: 
            current_content.append(line)
        elif current_main_title and not current_sub_title:
            current_content.append(line)
    
    # Simpan chunk terakhir
    if current_sub_title and current_content:
        content_text = ' '.join([l.strip() for l in current_content if l. strip()])
        content_text = ' '.join(content_text.split())
        
        if content_text:
            chunks. append({
                'chunk_id': chunk_id,
                'main_header_number': current_main_number,
                'main_header_title': current_main_title,
                'sub_header_title': current_sub_title,
                'content': content_text
            })
    elif current_main_title and current_content and not current_sub_title:
        content_text = ' '.join([l.strip() for l in current_content if l. strip()])
        content_text = ' '.join(content_text.split())
        
        if content_text:
            chunks. append({
                'chunk_id': chunk_id,
                'main_header_number': current_main_number,
                'main_header_title': current_main_title,
                'sub_header_title': None,
                'content': content_text
            })
    
    # Simpan intro
    if intro_content: 
        content_text = ' '.join([l.strip() for l in intro_content if l.strip()])
        content_text = ' '.join(content_text.split())
        
        if content_text: 
            chunks.insert(0, {
                'chunk_id': 0,
                'main_header_number': '0',
                'main_header_title': 'Pendahuluan',
                'sub_header_title':  None,
                'content': content_text
            })
    
    # Re-index
    for i, chunk in enumerate(chunks):
        chunk['chunk_id'] = i + 1
    
    df = pd.DataFrame(chunks)
    print(f"✅ Berhasil membuat {len(df)} chunks (per sub header)")
    return df

In [97]:
df_granular = chunk_by_sub_headers(cleaned_text)
df_granular

📑 Chunking per sub header (granular)...
✅ Berhasil membuat 12 chunks (per sub header)


,chunk_id,main_header_number,main_header_title,sub_header_title,content
0,1,0,Pendahuluan,None,PROPOSAL BISNIS STARTUP [NAMA STARTUP ANDA] Di...
1,2,3,Nama Anggota 2 - NIM - Prodi,None,KOTA BANDUNG TAHUN 2025 DAFTAR ISI
2,3,5,LAMPIRAN,None,BAB I: PENDAHULUAN
3,4,1,1 Latar Belakang Usaha,None,Petunjuk Pengisian: Jelaskan masalah utama yan...
4,5,2,1 Noble Purpose (Tujuan Mulia),None,Petunjuk Pengisian: Apa tujuan mulia dari bisn...
5,6,2,2 Identifikasi Konsumen Potensial,None,Petunjuk Pengisian: Siapa target pasar spesifi...
6,7,2,3 Produk Inovatif,None,Petunjuk Pengisian: Jelaskan detail produk ata...
7,8,2,4 Strategi Pemasaran,None,Petunjuk Pengisian: Bagaimana cara Anda menjan...
8,9,2,5 Sumber Daya yang Dimiliki,None,Petunjuk Pengisian: Jelaskan komposisi tim (ha...
9,10,3,1 Laporan/Proyeksi Keuangan,None,"Petunjuk Pengisian: Jika usaha sudah berjalan,..."


In [98]:
# Preview struktur
print("📋 STRUKTUR DOKUMEN (per sub header):")
print("="*80)
for idx, row in df_granular.iterrows():
    if pd.isna(row['sub_header_title']):
        print(f"\n{row['main_header_number']}. {row['main_header_title']}")
        print(f"   Content: {row['content'][:100]}...")
    else:
        print(f"   - {row['sub_header_title']}")
        print(f"     └─ {row['content'][:100]}...")


📋 STRUKTUR DOKUMEN (per sub header):

0. Pendahuluan
   Content: PROPOSAL BISNIS STARTUP [NAMA STARTUP ANDA] Diajukan Untuk: Program Inkubasi Startup - Universitas K...

3. Nama Anggota 2 - NIM - Prodi
   Content: KOTA BANDUNG TAHUN 2025 DAFTAR ISI...

5. LAMPIRAN
   Content: BAB I: PENDAHULUAN...

1. 1 Latar Belakang Usaha
   Content: Petunjuk Pengisian: Jelaskan masalah utama yang melatarbelakangi terbentuknya usaha ini. Sertakan da...

2. 1 Noble Purpose (Tujuan Mulia)
   Content: Petunjuk Pengisian: Apa tujuan mulia dari bisnis ini selain mencari keuntungan? Dampak positif apa y...

2. 2 Identifikasi Konsumen Potensial
   Content: Petunjuk Pengisian: Siapa target pasar spesifik Anda? Jelaskan demografi, psikografi, atau perilaku ...

2. 3 Produk Inovatif
   Content: Petunjuk Pengisian: Jelaskan detail produk atau jasa Anda. Apa keunikannya (Unique Value Proposition...

2. 4 Strategi Pemasaran
   Content: Petunjuk Pengisian: Bagaimana cara Anda menjangkau pelanggan? Jelaskan strateg

## 6. Chunking per Main Header (Gabungan)

In [99]:
def chunk_by_main_headers(text: str) -> pd.DataFrame:
    """
    Chunking per main header - Setiap main header + semua sub headernya jadi 1 chunk
    """
    print("📑 Chunking per main header (gabungan)...")
    
    lines = text.split('\n')
    chunks = []
    chunk_id = 1
    
    main_header_pattern = r'^(\d+)\.\s*(.+?)(?:\s*:)?$'
    
    current_main_number = None
    current_main_title = None
    current_content = []
    
    intro_content = []
    first_main_header_found = False
    
    for i, line in enumerate(lines):
        line_stripped = line.strip()
        
        # Cek main header
        main_match = re.match(main_header_pattern, line_stripped)
        if main_match: 
            # Simpan chunk sebelumnya
            if current_main_number and current_content:
                content_text = ' '.join([l.strip() for l in current_content if l.strip()])
                content_text = ' '.join(content_text.split())
                
                if content_text: 
                    chunks.append({
                        'chunk_id': chunk_id,
                        'main_header_number': current_main_number,
                        'main_header_title': current_main_title,
                        'content': content_text
                    })
                    chunk_id += 1
            
            # Update main header baru
            current_main_number = main_match.group(1)
            current_main_title = main_match.group(2).strip()
            current_content = []
            first_main_header_found = True
            continue
        
        # Tambahkan line ke content
        if not first_main_header_found:
            intro_content.append(line)
        else:
            current_content.append(line)
    
    # Simpan chunk terakhir
    if current_main_number and current_content:
        content_text = ' '.join([l. strip() for l in current_content if l.strip()])
        content_text = ' '.join(content_text.split())
        
        if content_text: 
            chunks.append({
                'chunk_id': chunk_id,
                'main_header_number': current_main_number,
                'main_header_title':  current_main_title,
                'content': content_text
            })
    
    # Simpan intro
    if intro_content:
        content_text = ' '.join([l.strip() for l in intro_content if l.strip()])
        content_text = ' '.join(content_text.split())
        
        if content_text: 
            chunks.insert(0, {
                'chunk_id':  0,
                'main_header_number': '0',
                'main_header_title':  'Pendahuluan',
                'content': content_text
            })
    
    # Re-index
    for i, chunk in enumerate(chunks):
        chunk['chunk_id'] = i + 1
    
    df = pd.DataFrame(chunks)
    print(f"✅ Berhasil membuat {len(df)} chunks (per main header)")
    return df

In [100]:
df_combined = chunk_by_main_headers(cleaned_text)
df_combined

📑 Chunking per main header (gabungan)...
✅ Berhasil membuat 12 chunks (per main header)


,chunk_id,main_header_number,main_header_title,content
0,1,0,Pendahuluan,PROPOSAL BISNIS STARTUP [NAMA STARTUP ANDA] Di...
1,2,3,Nama Anggota 2 - NIM - Prodi,KOTA BANDUNG TAHUN 2025 DAFTAR ISI
2,3,5,LAMPIRAN,BAB I: PENDAHULUAN
3,4,1,1 Latar Belakang Usaha,Petunjuk Pengisian: Jelaskan masalah utama yan...
4,5,2,1 Noble Purpose (Tujuan Mulia),Petunjuk Pengisian: Apa tujuan mulia dari bisn...
5,6,2,2 Identifikasi Konsumen Potensial,Petunjuk Pengisian: Siapa target pasar spesifi...
6,7,2,3 Produk Inovatif,Petunjuk Pengisian: Jelaskan detail produk ata...
7,8,2,4 Strategi Pemasaran,Petunjuk Pengisian: Bagaimana cara Anda menjan...
8,9,2,5 Sumber Daya yang Dimiliki,Petunjuk Pengisian: Jelaskan komposisi tim (ha...
9,10,3,1 Laporan/Proyeksi Keuangan,"Petunjuk Pengisian: Jika usaha sudah berjalan,..."


In [101]:
# Preview struktur
print("📋 STRUKTUR DOKUMEN (per main header):")
print("="*80)
for idx, row in df_combined.iterrows():
    print(f"\n{row['main_header_number']}. {row['main_header_title']}")
    print(f"   Word count: {len(row['content']. split())}")
    print(f"   Preview: {row['content'][:150]}...")

📋 STRUKTUR DOKUMEN (per main header):

0. Pendahuluan
   Word count: 20
   Preview: PROPOSAL BISNIS STARTUP [NAMA STARTUP ANDA] Diajukan Untuk: Program Inkubasi Startup - Universitas Komputer Indonesia [Logo Unikom] Disusun Oleh (Tim)...

3. Nama Anggota 2 - NIM - Prodi
   Word count: 6
   Preview: KOTA BANDUNG TAHUN 2025 DAFTAR ISI...

5. LAMPIRAN
   Word count: 3
   Preview: BAB I: PENDAHULUAN...

1. 1 Latar Belakang Usaha
   Word count: 25
   Preview: Petunjuk Pengisian: Jelaskan masalah utama yang melatarbelakangi terbentuknya usaha ini. Sertakan data atau fakta pendukung mengapa masalah ini pentin...

2. 1 Noble Purpose (Tujuan Mulia)
   Word count: 21
   Preview: Petunjuk Pengisian: Apa tujuan mulia dari bisnis ini selain mencari keuntungan? Dampak positif apa yang ingin diberikan kepada masyarakat atau lingkun...

2. 2 Identifikasi Konsumen Potensial
   Word count: 25
   Preview: Petunjuk Pengisian: Siapa target pasar spesifik Anda? Jelaskan demografi, psikografi, atau perilaku 

## 7. Pilih Opsi & Final Polish

In [103]:
# PILIH SALAH SATU: 
df_final = df_granular  # Jika mau per sub header (lebih detail)
# df_final = df_combined  # Jika mau per main header (lebih ringkas)

print(f"✅ Dipilih:  {'GRANULAR (per sub header)' if 'sub_header_title' in df_final.columns else 'COMBINED (per main header)'}")

✅ Dipilih:  GRANULAR (per sub header)


In [104]:
def final_polish(df: pd.DataFrame) -> pd.DataFrame:
    """
    Polish final untuk memastikan content benar-benar bersih
    """
    df = df.copy()
    
    def polish_content(text: str) -> str:
        if pd.isna(text):
            return text
        
        # Hapus semua line breaks
        text = text.replace('\n', ' ')
        
        # Hapus multiple spaces
        text = re.sub(r'\s+', ' ', text)
        
        # Fix spasi sebelum tanda baca
        text = re.sub(r'\s+([.,;:!?])', r'\1', text)
        
        # Fix spasi setelah tanda baca
        text = re.sub(r'([.,;:!?])([A-Za-z0-9])', r'\1 \2', text)
        
        # Trim
        text = text.strip()
        
        return text
    
    df['content'] = df['content']. apply(polish_content)
    df['main_header_title'] = df['main_header_title'].apply(polish_content)
    
    if 'sub_header_title' in df.columns:
        df['sub_header_title'] = df['sub_header_title']. apply(polish_content)
    
    return df

In [105]:
df_final = final_polish(df_final)

# Tambahkan informasi
df_final['word_count'] = df_final['content'].apply(lambda x: len(x.split()))
df_final['char_count'] = df_final['content'].apply(lambda x: len(x))
df_final['source_file'] = pdf_file_path

print("✅ DataFrame final siap!")
print(f"   Total chunks: {len(df_final)}")
print(f"   Rata-rata kata/chunk: {df_final['word_count'].mean():.0f}")

df_final

✅ DataFrame final siap!
   Total chunks: 12
   Rata-rata kata/chunk: 21


,chunk_id,main_header_number,main_header_title,sub_header_title,content,word_count,char_count,source_file
0,1,0,Pendahuluan,None,PROPOSAL BISNIS STARTUP [NAMA STARTUP ANDA] Di...,20,151,Template Proposal.pdf
1,2,3,Nama Anggota 2 - NIM - Prodi,None,KOTA BANDUNG TAHUN 2025 DAFTAR ISI,6,34,Template Proposal.pdf
2,3,5,LAMPIRAN,None,BAB I: PENDAHULUAN,3,18,Template Proposal.pdf
3,4,1,1 Latar Belakang Usaha,None,Petunjuk Pengisian: Jelaskan masalah utama yan...,25,195,Template Proposal.pdf
4,5,2,1 Noble Purpose (Tujuan Mulia),None,Petunjuk Pengisian: Apa tujuan mulia dari bisn...,21,154,Template Proposal.pdf
5,6,2,2 Identifikasi Konsumen Potensial,None,Petunjuk Pengisian: Siapa target pasar spesifi...,25,199,Template Proposal.pdf
6,7,2,3 Produk Inovatif,None,Petunjuk Pengisian: Jelaskan detail produk ata...,27,208,Template Proposal.pdf
7,8,2,4 Strategi Pemasaran,None,Petunjuk Pengisian: Bagaimana cara Anda menjan...,24,194,Template Proposal.pdf
8,9,2,5 Sumber Daya yang Dimiliki,None,Petunjuk Pengisian: Jelaskan komposisi tim (ha...,32,241,Template Proposal.pdf
9,10,3,1 Laporan/Proyeksi Keuangan,None,"Petunjuk Pengisian: Jika usaha sudah berjalan,...",25,192,Template Proposal.pdf


## 8. Export ke File

In [ ]:
# Tentukan nama file
file_suffix = 'granular' if 'sub_header_title' in df_final.columns else 'combined'

# Export ke CSV
csv_filename = f'proposal_chunks_{file_suffix}.csv'
df_final.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"✅ Saved:  {csv_filename}")

# Export ke Excel
xlsx_filename = f'proposal_chunks_{file_suffix}.xlsx'
df_final.to_excel(xlsx_filename, index=False, engine='openpyxl')
print(f"✅ Saved: {xlsx_filename}")

# Export ke JSON
json_filename = f'proposal_chunks_{file_suffix}.json'
df_final. to_json(json_filename, orient='records', force_ascii=False, indent=2)
print(f"✅ Saved: {json_filename}")

print(f"\n📊 Total chunks exported: {len(df_final)}")

## 9. Statistik dan Ringkasan

In [106]:
print("="*80)
print("📊 STATISTIK FINAL")
print("="*80)
print(f"\n📄 File: {pdf_file_path}")
print(f"🎯 Mode: {'GRANULAR (per sub header)' if 'sub_header_title' in df_final.columns else 'COMBINED (per main header)'}")

print(f"\n✂️ Chunking:")
print(f"   Total chunks: {len(df_final)}")
print(f"   Rata-rata kata/chunk: {df_final['word_count']. mean():.0f}")
print(f"   Min kata: {df_final['word_count'].min()}")
print(f"   Max kata: {df_final['word_count'].max()}")

print(f"\n📋 DAFTAR CHUNKS:")
for idx, row in df_final. iterrows():
    if 'sub_header_title' in df_final.columns:
        sub_info = f" - {row['sub_header_title']}" if not pd.isna(row['sub_header_title']) else ""
        print(f"   [{row['chunk_id']}] {row['main_header_number']}. {row['main_header_title']}{sub_info} ({row['word_count']} kata)")
    else:
        print(f"   [{row['chunk_id']}] {row['main_header_number']}. {row['main_header_title']} ({row['word_count']} kata)")

print(f"\n💾 Output files:")
print(f"   ✅ {csv_filename}")
print(f"   ✅ {xlsx_filename}")
print(f"   ✅ {json_filename}")
print("\n" + "="*80)
print("🎉 SELESAI!")
print("="*80)

📊 STATISTIK FINAL

📄 File: Template Proposal.pdf
🎯 Mode: GRANULAR (per sub header)

✂️ Chunking:
   Total chunks: 12
   Rata-rata kata/chunk: 21
   Min kata: 3
   Max kata: 32

📋 DAFTAR CHUNKS:
   [1] 0. Pendahuluan (20 kata)
   [2] 3. Nama Anggota 2 - NIM - Prodi (6 kata)
   [3] 5. LAMPIRAN (3 kata)
   [4] 1. 1 Latar Belakang Usaha (25 kata)
   [5] 2. 1 Noble Purpose (Tujuan Mulia) (21 kata)
   [6] 2. 2 Identifikasi Konsumen Potensial (25 kata)
   [7] 2. 3 Produk Inovatif (27 kata)
   [8] 2. 4 Strategi Pemasaran (24 kata)
   [9] 2. 5 Sumber Daya yang Dimiliki (32 kata)
   [10] 3. 1 Laporan/Proyeksi Keuangan (25 kata)
   [11] 3. 2 Rencana Anggaran Belanja (RAB) Produksi (22 kata)
   [12] 4. 1 Kesimpulan (19 kata)

💾 Output files:


NameError: name 'csv_filename' is not defined